In [17]:
from bs4 import BeautifulSoup
import numpy as np
import os
import scipy.stats as s

In [18]:
# choose arguments
average_last_n = 5 # number of final RLCT values averaged from 1 epoch

In [19]:
# converts javascript code from plotly plot in html to numpy arrays
def jscript_to_array(js_code, start_string, end_string):
    start_len = len(start_string)

    start_index = js_code.find(start_string)
    if start_index != -1: 
        start_index += start_len
        rlct_end_index = js_code.find(end_string, start_index)
        array_string = js_code[start_index:rlct_end_index]
        numpy_array = np.array(eval(array_string))
        return numpy_array
    return np.array([])

In [20]:
def data_from_file(directory, filename, average_last_n):
    html_file_path = directory + '/' + filename

    # Load the HTML file
    with open(html_file_path, "r") as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html.parser")

    # Find all <script> tags
    script_tags = soup.find_all("script")

    # helper string initialisation
    sgd_rlct_start_string = 'sgd RLCT","x":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],"y":'
    ngd_rlct_start_string = 'ngd RLCT","x":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],"y":'
    rlct_end_string = ',"type":"scatter"'

    sgd_matrix = []
    ngd_matrix = []

    # Iterate over each <script> tag
    for script_tag in script_tags:
        # Extract the JavaScript code
        js_code = script_tag.string

        # Check if the script contains Plotly initialization code
        if js_code and "Plotly.newPlot" in js_code:
            # sgd
            rlct_array = jscript_to_array(js_code, sgd_rlct_start_string, rlct_end_string)
            if (rlct_array.size > 0):
                sgd_matrix.append(rlct_array)


            #ngd
            rlct_array = jscript_to_array(js_code, ngd_rlct_start_string, rlct_end_string)
            if (rlct_array.size > 0):
                ngd_matrix.append(rlct_array)

    sgd_matrix = np.array(sgd_matrix)
    ngd_matrix = np.array(ngd_matrix)

    sgd_matrix_average = sgd_matrix[:,-average_last_n:].mean(axis = 1)
    ngd_matrix_average = ngd_matrix[:,-average_last_n:].mean(axis = 1)

    return sgd_matrix_average, ngd_matrix_average

    


In [21]:
# Get the experiment directory
directory = "ngd_sgd"
count = 0

sgd_rlct_values = []
ngd_rlct_values = []

# Loop through the files in the current working directory
for filename in os.listdir(directory):
    # Check if the filename contains the word "seed"
    if "seed" in filename:
        count += 1
        sgd_matrix_average, ngd_matrix_average = data_from_file(directory, filename, average_last_n)
        sgd_rlct_values.append(sgd_matrix_average)
        ngd_rlct_values.append(ngd_matrix_average)

print("number of files: ", count)

number of files:  30


In [23]:
# experiment format: [rlct1, rlct2, rlct3, rlct4, rlct5, rlct6]
# RLCTs obtained from the following settings:
# "hidden_nodes" : [64, 128, 256, 64, 128, 256], "hidden_layers" : [1, 1, 1, 2, 2, 2],

# calculate the std and mean of the 2 distrbutions
sgd_rlct_values = np.array(sgd_rlct_values)
ngd_rlct_values = np.array(ngd_rlct_values)

sgd_rlct_std = np.std(sgd_rlct_values, axis=0)
ngd_rlct_std = np.std(ngd_rlct_values, axis=0)

sgd_rlct_mean = np.mean(sgd_rlct_values, axis=0)
ngd_rlct_mean = np.mean(ngd_rlct_values, axis=0)

print("sgd rlct std: ", sgd_rlct_std)
print("ngd rlct std: ", ngd_rlct_std)
print()
print("sgd rlct mean: ", sgd_rlct_mean)
print("ngd rlct mean: ", ngd_rlct_mean)

sgd rlct std:  [0.47802885 0.52507816 0.52439516 0.53304194 0.43003263 0.53077106]
ngd rlct std:  [0.62964714 0.65414845 0.73289246 0.59286157 1.22941648 1.01080784]

sgd rlct mean:  [0.47802885 0.52507816 0.52439516 0.53304194 0.43003263 0.53077106]
ngd rlct mean:  [0.62964714 0.65414845 0.73289246 0.59286157 1.22941648 1.01080784]


In [24]:
# hypothesis test: rlct_sgd < rlct_sgd

# test 1: one tailed Welch's t-test 
print("test 1: one tailed Welch's t-test")
print("This test compares if NGD's distribution is higher than SGD's distribution")
result = s.ttest_ind(sgd_rlct_values, ngd_rlct_values, axis=0, equal_var=False, alternative="less")
print("Welch's t-test pvalues: ", result.pvalue)
print()

# test 2: one-tailed test using seed 1
print("test 2: one-tailed z-test uing seed 1")
print("This test checks if NGD values for seed 1 are higher than SGD")
z_score = (np.array([22.8, 31.3, 36.8, 33, 43.5, 57.4]) - sgd_rlct_mean)/sgd_rlct_std
print("z scores: ", z_score)
print("p values for ngd seed1: ",1 - s.norm.cdf(z_score))

test 1: one tailed Welch's t-test
This test compares if NGD's distribution is higher than SGD's distribution
Welch's t-test pvalues:  [8.49122227e-50 9.53869955e-62 9.01733633e-59 9.19883720e-50
 1.05553954e-32 1.15606202e-44]

test 2: one-tailed z-test uing seed 1
This test checks if NGD values for seed 1 are higher than SGD
z scores:  [13.5415131  23.33043818 25.5459749  14.56745867 24.98590093 25.83921972]
p values for ngd seed1:  [0. 0. 0. 0. 0. 0.]
